<a href="https://colab.research.google.com/github/EddyEjembi/EdgeML-FOMO/blob/main/Model_Quantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install onnx tensorflow onnxruntime tf2onnx onnx-tf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.2 MB/s eta 0:00:00


##Fixing ONNX Bug
Source: https://stackoverflow.com/a/76941526/20191333

In [2]:
import onnx

onnx_model = onnx.load("/content/fomo_tf_model.onnx")
print("Model Inputs: ", [inp.name for inp in onnx_model.graph.input])

Model Inputs:  ['input.1']


In [3]:
import onnx
from onnx import helper

onnx_model = onnx.load("/content/fomo_tf_model.onnx")

# Define a mapping from old names to new names
name_map = {"input.1": "input_1"}

# Initialize a list to hold the new inputs
new_inputs = []

# Iterate over the inputs and change their names if needed
for inp in onnx_model.graph.input:
    if inp.name in name_map:
        # Create a new ValueInfoProto with the new name
        new_inp = helper.make_tensor_value_info(name_map[inp.name],
                                                inp.type.tensor_type.elem_type,
                                                [dim.dim_value for dim in inp.type.tensor_type.shape.dim])
        new_inputs.append(new_inp)
    else:
        new_inputs.append(inp)

# Clear the old inputs and add the new ones
onnx_model.graph.ClearField("input")
onnx_model.graph.input.extend(new_inputs)

# Go through all nodes in the model and replace the old input name with the new one
for node in onnx_model.graph.node:
    for i, input_name in enumerate(node.input):
        if input_name in name_map:
            node.input[i] = name_map[input_name]

# Save the renamed ONNX model
onnx.save(onnx_model, 'fomo-tf-model-new.onnx')

##Convert ONNX Model to TensorFlow

In [4]:
import onnx
from onnx_tf.backend import prepare

# Load the ONNX model
onnx_model = onnx.load("/content/fomo-tf-model-new.onnx")

# Convert the ONNX model to TensorFlow
tf_rep = prepare(onnx_model)

# Export the TensorFlow model to a SavedModel format
tf_rep.export_graph("tf_model")


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


##Quantize Model to TensorFlow Lite Micro

In [7]:
import tensorflow as tf

# Load the TensorFlow SavedModel
saved_model_dir = "tf_model"

# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

# Save the TensorFlow Lite model
with open("model.tflite", "wb") as f:
    f.write(tflite_model)


## Convert TFLite Micro model to ONNX

In [16]:
! python -m tf2onnx.convert --tflite /content/model.tflite --output lite-model.onnx

2024-07-02 21:26:02.891281: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 21:26:02.891357: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-02 21:26:02.892736: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-02 21:26:03.892509: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2

In [21]:
from PIL import Image
import numpy as np

def process_image(image_path):
    # Load the image
    image = Image.open(image_path).convert('RGB')

    # Resize the image
    image = image.resize((224, 224))

    # Convert the image to a numpy array
    image_array = np.array(image, dtype=np.float32) / 255.0

    # Normalize the image
    image_array = (image_array - 0.5) / 0.5

    # Transpose the array to have the shape [3, 224, 224]
    image_array = np.transpose(image_array, (2, 0, 1))

    # Add a batch dimension
    image_array = np.expand_dims(image_array, axis=0)

    return image_array

In [22]:
import onnxruntime as ort

# Function to perform inference
def run_inference(model_path, image_path):
    # Preprocess the image
    input_tensor = process_image(image_path)

    # Load the ONNX model
    session = ort.InferenceSession(model_path)

    # Get the input name for the ONNX model
    input_name = session.get_inputs()[0].name

    # Get the output name for the ONNX model
    output_name = session.get_outputs()[0].name

    # Perform inference
    result = session.run(None, {input_name: input_tensor})

    return result

In [25]:
# Path to your ONNX model
model_path = '/content/lite-model.onnx'

# Path to your image
image_path = '/content/banana_36.jpg'

# Run inference
result = run_inference(model_path, image_path)

print("Results: ",result)
print("Predicted Class: ", np.argmax(result)) # 0 - apple, 1 - bannana, 2 - orange, 3 - others

Results:  [array([[-0.8363082 ,  1.2549214 ,  0.29177618, -0.59465414]],
      dtype=float32)]
Predicted Class:  1
